# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
buy_each = data.groupby(['CustomerID','ProductName'],as_index=False).agg({'Quantity':'sum'})
buy_each

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
cus_matrix = buy_each.pivot_table(index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
cus_matrix


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0
Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
Arizona - Green Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
Artichokes - Jerusalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(cus_matrix.T, 'euclidean'))), 
                         index=cus_matrix.columns, columns=cus_matrix.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
results_33 = distances[33].sort_values(ascending=False)[1:6]
results_33

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:

names_33 = list(results_33.keys())
print(names_33)

similar = buy_each.loc[buy_each.CustomerID.isin(names_33)]
similar.head()


[264, 3535, 3317, 2503, 3305]


,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
ranked = similar.groupby(['ProductName'],as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
ranked.reset_index(drop=True)

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3
5,"Pepper - Black, Whole",3
6,Chicken - Soup Base,2
7,Cod - Black Whole Fillet,2
8,Chocolate - Dark,2
9,Raspberries - Fresh,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
customer = data[data['CustomerID']==33]['ProductName']
customer[:5]

ranked_not_buyed_33 = ranked.loc[~ranked['ProductName'].isin(customer)]
ranked_not_buyed_33.head(5)

filtered_matrix = pd.merge(cus_matrix, ranked_not_buyed_33, on='ProductName')
filtered_matrix.head()
print('shape filtered_matrix:',filtered_matrix.shape)

527      Beer - Original Organic Lager
1017              Lamb - Pieces, Diced
3012    French Pastry - Mini Chocolate
3476               Beer - Rickards Red
3795                           Mangoes
Name: ProductName, dtype: object

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
43,Chicken - Soup Base,2


,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,1
2,Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0,1
3,Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0,1
4,Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,1


shape filtered_matrix: (187, 1002)


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
from IPython.display import clear_output

def get_5_close(cus_id, distances):
    results_5 = distances[cus_id].sort_values(ascending=False)[1:6]
    return list(results_5.keys())

def get_ranked_by5(cus_id, distances, buy_each):
    close_5 = get_5_close(cus_id, distances)
    similar = buy_each.loc[buy_each.CustomerID.isin(close_5)]
    ranked = similar.groupby(['ProductName'],as_index=False).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
    ranked.reset_index(drop=True)
    return ranked
    
def get_ranked_not_buyed_by(customer_id, ranked, data):
    buy_by_customer = data[data['CustomerID']==customer_id]['ProductName']
    ranked_not = ranked.loc[~ranked['ProductName'].isin(customer)]
    return ranked_not

def get_filtered_matrix(cus_id, cus_matrix, ranked):
    return pd.merge(cus_matrix,get_ranked_not_buyed_by(cus_id, ranked, data), on='ProductName')[:5]

def get_dic(filtered):
        alist = filtered['ProductName'].values.tolist()
        di = {}
        for i, item in enumerate(alist):
            di['product'+str(i+1)] = item
        return di

def output(i, force = False):
    global res, prev, L
    res = round(i/L*100,2)
    if res - prev > 0.1 or force:
        clear_output()
        display('Completed: '+str(res)+ '%')
        prev = res
            
def get_recommendations(data, distance_method = 'euclidean'):
    dic = {}
    customers_id = data['CustomerID'].values.tolist()
    buy_each = data.groupby(['CustomerID','ProductName'],as_index=False).agg({'Quantity':'sum'})
    cus_matrix = buy_each.pivot_table(index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
    distances = pd.DataFrame(1/(1 + squareform(pdist(cus_matrix.T, distance_method))), 
                         index=cus_matrix.columns, columns=cus_matrix.columns)
    
    global L, prev, res
    res = 0
    prev = 0
    L = len(customers_id)
    for i, cus_id in enumerate(customers_id):
        
        output(i)
        ranked = get_ranked_by5(cus_id, distances, buy_each)
        ranked_not_buyed = get_ranked_not_buyed_by(cus_id, ranked, data)
        filtered = get_filtered_matrix(cus_id, cus_matrix, ranked_not_buyed)
        dic[cus_id] = get_dic(filtered)
    output(i, force=True)
    return dic
              


In [12]:
dict_recomm_euclidean = get_recommendations(data,'euclidean')

'Completed: 100.0%'

In [13]:
dict_recomm_euclidean[33]

{'product1': 'Anchovy Paste - 56 G Tube',
 'product2': 'Appetizer - Mushroom Tart',
 'product3': 'Appetizer - Sausage Rolls',
 'product4': 'Apricots - Halves',
 'product5': 'Apricots Fresh'}

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [14]:
def get_dataframe(dict_recomm):
    colnames = ['product1','product2','product3','product4','product5']
    result = pd.DataFrame(dict_recomm).transpose()
    result.columns = colnames
    result = result.reset_index()
    result = result.rename(columns={'index':'CustomerID'})
    return result

In [15]:
df_euclidean = get_dataframe(dict_recomm_euclidean)
df_euclidean.head()

,CustomerID,product1,product2,product3,product4,product5
0,61288,Appetizer - Mushroom Tart,Apricots - Halves,Arizona - Green Tea,Bacardi Breezer - Tropical,Bagel - Plain
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Arizona - Green Tea,Bacardi Breezer - Tropical,Baking Powder
2,40094,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem
3,23548,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves
4,78981,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
dict_recomm_cosine = get_recommendations(data,'cosine')

'Completed: 100.0%'

In [17]:
dict_recomm_cosine[33]
df_cosine = get_dataframe(dict_recomm_cosine)
df_cosine.head()

{'product1': 'Appetizer - Sausage Rolls',
 'product2': 'Apricots - Halves',
 'product3': 'Baking Powder',
 'product4': 'Banana Turning',
 'product5': 'Bandage - Fexible 1x3'}

,CustomerID,product1,product2,product3,product4,product5
0,61288,Anchovy Paste - 56 G Tube,Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots Fresh,Arizona - Green Tea
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Sausage Rolls,Apricots Fresh,Bagel - Plain
2,40094,Anchovy Paste - 56 G Tube,Appetizer - Mushroom Tart,Apricots - Halves,Artichokes - Jerusalem,Bagel - Plain
3,23548,Anchovy Paste - 56 G Tube,Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves,Apricots Fresh
4,78981,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Artichokes - Jerusalem,Bagel - Plain
